**Ссылка**, на источник текста

In [1]:
DATA_URL = "http://lib.ru/LITRA/LERMONTOW/geroi.txt"

Устанавливаем библиотеки

In [2]:
import warnings
warnings.filterwarnings('ignore')
!pip install -q git+https://github.com/dvolchek/rnnmorph.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 519.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Создаём объект морфологического анализатора `RNNMorph`

In [3]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [4]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [5]:
raw_text[:200]

'<html><head><title>Михаил Лермонтов. Герой нашего времени</title></head><body><pre><div align=right><form action=/LITRA/LERMONTOW/geroi.txt><select name=format><OPTION VALUE="_Contents">Содержание<OPT'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [7]:
cleaned_text[:200]

'Михаил Лермонтов. Герой нашего времениСодержаниеFine HTMLPrinted versiontxt(Word,КПК)Lib.ru html\nМихаил Лермонтов. Герой нашего времени\n\n\n     Во всякой книге предисловие есть первая и вместе с тем по'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


"A total of 3194 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [9]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent]
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться

50/50 [==============================] - 53s 1s/step


sentences: 100%|██████████| 3194/3194 [00:00<00:00, 8911.89it/s]


[['fievre', 'lente', '-', 'медленный', 'горячка', '(', 'франц', '.', ')', '.']]

In [10]:
predictions = [[token for token in sentence if token.isalpha()] for sentence in predictions]

In [22]:
print(f'Количество предложений: {len(predictions)}')

non_uniq_tokens = [word for sentence in predictions for word in sentence]
print(f'Количество буквенных токенов: {len(non_uniq_tokens)}')

Количество предложений: 3194
Количество буквенных токенов: 41655


## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам.

In [13]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['и', 'в', 'во', 'не', 'что']

In [21]:
top_token_frequency = 100

top100_tokens = FreqDist(non_uniq_tokens).most_common(top_token_frequency)
clean_tokens_top100 = [token for token in top100_tokens if token[0] not in STOPWORDS]
len(clean_tokens_top100)/len(top100_tokens)

0.45

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

In [20]:
max_token_frequency = 50

i = 0
while top100_tokens[i][1] > max_token_frequency:
  count = i
  i += 1
print(f'Количество токенов, встречающихся строго больше {max_token_frequency} раз: {count + 1}')

Количество токенов, встречающихся строго больше 50 раз: 91
